In [54]:
import json
import pandas as pd
import numpy as np

In [55]:
SOILPOINTS_FILE = "Africa_Soil_Pts.csv"

In [56]:
# Initialize Caches
SOILPOINTS_DF = None
ADMIN_REGIONS_CACHE = {}

In [57]:
def get_soilpoints():
    global SOILPOINTS_DF
    if SOILPOINTS_DF is None:
        df = pd.read_csv(SOILPOINTS_FILE, header=0)
        df.lat = np.round(df.lat, decimals = 8)
        df.lon = np.round(df.lon, decimals = 8)       
        df = df.set_index(['lon', 'lat'])
        SOILPOINTS_DF = df
    return SOILPOINTS_DF

In [58]:
def get_admin_regions_for_soil_point(lat, lng):
    # Reduce precision
    lat = round(lat, 8)
    lng = round(lng, 8)
    
    if (lat,lng) in ADMIN_REGIONS_CACHE:
        return ADMIN_REGIONS_CACHE[(lat,lng)]

    df = get_soilpoints()
    row = df.loc[(lng, lat)]
    admin_regions = {
        "country": row.country,
        "admin1": row.admin1,
        "admin2": row.admin2,
        "admin3": row.admin3
    }
    ADMIN_REGIONS_CACHE[(lat,lng)] = admin_regions
    return admin_regions

In [59]:
def get_resources(resource_file):
    resources = []
    with open(resource_file, "r") as rf:
        data = json.load(rf)
        for res in data["resources"]:
            geo = res["resource_metadata"]["spatial_coverage"]["value"]
            admin_regions = get_admin_regions_for_soil_point(float(geo["y"]), float(geo["x"]))
            resources.append({
                "filename": res["resource_name"],
                "country": admin_regions["country"],
                "admin1": admin_regions["admin1"],
                "admin2": admin_regions["admin2"],
                "admin3": admin_regions["admin3"],                
            })
    return resources

In [64]:
def create_soilfile_mapping():
    df = pd.DataFrame(columns=[
        'country', 'admin1', 'admin2', 'admin3', 'filename'
    ])
    countries = ["Kenya", "Ethiopia", "SouthSudan", "Sudan", "Uganda", "Somalia", "Djibouti", "Eritrea"]
    for country in countries:
        print(country)
        resources = get_resources("Resources/"+country+"-Resources.json")
        for resource in resources:
            df = df.append(resource, ignore_index = True)

    df.set_index(['country', 'admin1', 'admin2', 'admin3'], inplace=True)
    df.to_csv('HOAResources.csv')

In [65]:
create_soilfile_mapping()

Kenya
Ethiopia
SouthSudan
Sudan
Uganda
Somalia
Djibouti
Eritrea
